# AutoML

## Imports

In [6]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
import json
import pandas as pd
from pathlib import Path
from azure.ai.ml import MLClient

## Setting Up ML Client

In [7]:
# 🔐 Authenticate and connect to Azure ML workspace
credential = DefaultAzureCredential()
ml_client = MLClient.from_config(credential=credential)

# ✅ Confirm connection
print("\n✅ Azure ML Workspace Loaded Successfully\n")
print(f"🔑 Subscription ID : {ml_client.subscription_id}")
print(f"📦 Resource Group  : {ml_client.resource_group_name}")
print(f"🧠 Workspace Name  : {ml_client.workspace_name}\n")

Found the config file in: /config.json



✅ Azure ML Workspace Loaded Successfully

🔑 Subscription ID : 03571d3d-f675-492f-8346-8b04c78ed80f
📦 Resource Group  : student-performance-rg
🧠 Workspace Name  : student-performance-ws



## Grabbing the Name of the Storage Account

In [11]:
from azure.mgmt.storage import StorageManagementClient

subscription_id = ml_client.subscription_id  
resource_group = ml_client.resource_group_name

storage_client = StorageManagementClient(credential, subscription_id)

# List storage accounts in the resource group
accounts = list(storage_client.storage_accounts.list_by_resource_group(resource_group))

# Check if any accounts exist
if not accounts:
    raise Exception("❌ No storage accounts found in this resource group.")

# Grab the first one
storage_account_name = accounts[0].name

# ✅ Optional: print them
print("\n📦 Storage Accounts in Resource Group:")
for account in accounts:
    print("•", account.name)

print(f"\n✅ Using storage account named: {storage_account_name}")


📦 Storage Accounts in Resource Group:
• studperfstor

✅ Using storage account named: studperfstor


In [13]:
import json
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobClient

# === Your values ===
container_name = "student-math-project-files"     # CONTAINER_NAME
relative_path = "config.json"                     # CONFIG_BLOB_NAME

# === Build blob URL and load JSON ===
blob_url = f"https://{storage_account_name}.blob.core.windows.net/{container_name}/{relative_path}"
blob_client = BlobClient.from_blob_url(blob_url, credential=credential)

# Download and parse JSON
json_bytes = blob_client.download_blob().readall()
json_data = json.loads(json_bytes)

# ✅ Pretty print JSON
print(json.dumps(json_data, indent=2))

{
  "metadata": {
    "created_at": "2025-04-16T14:10:33+0200",
    "created_year": "2025",
    "created_month": "April",
    "created_day": "16",
    "created_time": "14:10:33 CEST"
  },
  "azure": {
    "subscription_id": "03571d3d-f675-492f-8346-8b04c78ed80f",
    "resource_group": "student-performance-rg",
    "location": "norwayeast",
    "storage_account_name": "studperfstor",
    "storage_container_uri": "https://studperfstor.blob.core.windows.net/student-math-project-files",
    "managed_identity_principal_id": "46f12569-486b-4ab0-9fa8-de56d3d65640",
    "auth_mode": "managed_identity"
  },
  "workspace": {
    "workspace_name": "student-performance-ws",
    "storage_account_id": "/subscriptions/03571d3d-f675-492f-8346-8b04c78ed80f/resourceGroups/student-performance-rg/providers/Microsoft.Storage/storageAccounts/studperfstor",
    "key_vault_id": "/subscriptions/03571d3d-f675-492f-8346-8b04c78ed80f/resourceGroups/student-performance-rg/providers/Microsoft.KeyVault/vaults/studen